In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [ ]:
def build_lenet5():
    model = models.Sequential()
    model.add(layers.Conv2D(6, (5, 5), activation='relu', input_shape=(32, 32, 1))) ##LeNet requires to have input 32x32 based on its architecture
    model.add(layers.AveragePooling2D(pool_size=(2,2)))
    model.add(layers.Conv2D(16, (5, 5), activation='relu'))
    model.add(layers.AveragePooling2D(pool_size=(2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(84, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))
    return model

In [ ]:
lenet5_model = build_lenet5()
lenet5_model.summary()

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()


train_images = tf.pad(train_images, [[0, 0], [2, 2], [2, 2]])
train_images = tf.cast(train_images, tf.float32) / 255.0
test_images = tf.pad(test_images, [[0, 0], [2, 2], [2, 2]])
test_images = tf.cast(test_images, tf.float32) / 255.0

train_images.shape

In [ ]:

# Plotting the image
plt.imshow(train_images[0, :, :], cmap='gray')  # 'cmap' specifies that we want to see the image in grayscale
plt.title('First Image from Training Data')
plt.colorbar()
plt.show()


In [ ]:


train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)


#create batches
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)  # Shuffle and batch the data
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(32)  # Only batch the test data (no need to shuffle)


In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

In [ ]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = lenet5_model(images,training = True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, lenet5_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, lenet5_model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [77]:
epochs = 5

for epoch in range(epochs):
    train_loss.reset_state()
    train_accuracy.reset_state()

    for images, labels in train_dataset:
        train_step(images, labels)
        
    print(f'Epoch {epoch + 1}, '
          f'Loss: {train_loss.result()}, '
          f'Accuracy: {train_accuracy.result() * 100}')

2024-04-27 21:22:44.892153: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, Loss: 0.22310277819633484, Accuracy: 93.0999984741211


2024-04-27 21:23:43.397125: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Loss: 0.0722101703286171, Accuracy: 97.81999969482422


2024-04-27 21:24:41.900142: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 3, Loss: 0.049123164266347885, Accuracy: 98.50333404541016


2024-04-27 21:25:40.502036: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4, Loss: 0.03785302862524986, Accuracy: 98.77333068847656
Epoch 5, Loss: 0.030713137239217758, Accuracy: 99.06832885742188


2024-04-27 21:26:39.040603: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [79]:
lenet5_model.save("lenet5_model.keras")